Tenemos un dataset de pacientes con cancer de mama 

Cada paciente está representado por
- x: radio de la muestra (continua)
- z: textura de la muestra (continua)
- y: etiqueta de la muestra (0:maligno o 1:benigno)

El dataset tiene 569 pacientes, 212 con tumores malignos(0) y 357 tumores benignos (1)

In [ ]:
x, y = data_set['data'][:, :2], data_set['target']
fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
ax.scatter(x[y==0, 0], x[y==0, 1], c='k', s=10, marker='o', label='Sanos', alpha=0.5)
ax.scatter(x[y==1, 0], x[y==1, 1], c='k', s=10, marker='x', label='Cancer', alpha=0.5)
plt.legend();

Usando el teorema de Bayes podemos escribir la probabilidad de que el tumor sea benigno dada sus características (*posterior*) como

$$
p(y=1|x, z) = \frac{p(x, z|y=1) p(y=1)}{p(x, z)}
$$

y la probabilidad de que sea maligno como
$$
p(y=0|x, z) = \frac{p(x, z|y=0) p(y=0)}{p(x, z)}
$$

- Nos interesa saber si $p(y=0|x, z) > p(y=1|x, z)$
- La evidencia $p(x, z)$ es difícil de estimar 

Entonces

$$
\frac{p(y=1|x, z)}{p(y=0|x, z)} = \frac{p(x, z|y=1) p(y=1)}{p(x, z|y=0) p(y=0)}
$$

y los *priors*: $p(y=0) = \frac{212}{569} \approx 0.41$, $p(y=1) = \frac{357}{569} \approx 0.59$

Lo único que falta es la verosimilitud, asumiremos que

- las características son condicionalmente independientes
$$
p(x, z|y) = p(x|y) p(z|y)
$$
- las características son normales
$$
p(x|y) = \mathcal{N}(\mu_x, \sigma_x^2)
$$

Estos supuestos son lo que hacen al clasificador "ingenuo"

In [ ]:
# Probabilidades a priori
from collections import Counter
py = [Counter(y)[i]/len(y) for i in range(2)]
# Ajuste de verosimilitudes
dists = {}
for y_ in [0, 1]: # Para cada clase
    for x_ in [0, 1]: # para cada característica
        params = scipy.stats.norm.fit(x[y==y_, x_])
        dists[(y_, x_)] = scipy.stats.norm(loc=params[-2], scale=params[-1])

def likelihoods(x, z):
    pxzy0 = dists[(0, 0)].pdf(x)*dists[(0, 1)].pdf(z)
    pxzy1 = dists[(1, 0)].pdf(x)*dists[(1, 1)].pdf(z)
    return pxzy0, pxzy1, (pxzy1*py[1])/(pxzy0*py[0])

fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
for k, (label, marker) in enumerate(zip(['Sanos', 'Cancer'], ['o', 'x'])):
    ax.scatter(x[y==k, 0], x[y==k, 1], c='k', s=10, 
               marker=marker, label=label, alpha=0.5)

x_plot = np.linspace(np.amin(x[:, 0]), np.amax(x[:, 0]), num=500)
z_plot = np.linspace(np.amin(x[:, 1]), np.amax(x[:, 1]), num=500)
X, Z = np.meshgrid(x_plot, z_plot)
Y = likelihoods(X, Z)
ax.contourf(X, Z, Y[1] - Y[0], zorder=-1, cmap=plt.cm.RdBu, 
            vmin=-2e-2, vmax=2e-2, levels=20)
ax.set_xlim([np.amin(x_plot), np.amax(x_plot)])
ax.set_ylim([np.amin(z_plot), np.amax(z_plot)])
ax.set_xlabel('X'); ax.set_ylabel('Z');
plt.legend();

Decidimos entre sano y enfermo usando el cociente entre los posterior

$$
\frac{p(y=1|x, z)}{p(y=0|x, z)} > R
$$

Dos tipos de errores:
1. Predecir que está enfermo $\hat y=1$ cuando en realidad estaba sano $y=0$
1. Predecir que está sano $\hat y=0$ cuando en realidad estaba enfermo $y=1$

¿Las decisiones tienen igual riesgo ($R$=1) ? 

Podemos "calibrar" la importancia de las decisiones usando un $R$ mayor o menor que $1$

In [ ]:
ax.contour(X, Z, Y[2] > 1., zorder=-1, levels=[0]);
#ax.contour(X, Z, Y[2] > 0.5, zorder=-1, levels=[0]);
#ax.contour(X, Z, Y[2] > 0.1, zorder=-1, levels=[0]);

#### Resumen
- Cada clase es representada por una distribución Gaussiana
- Entrenamos cada Gaussiana por **máxima verosimilitud**
- El cociente entre los posterior nos da la clase más probable
- **Ventajas:** Fácil de entrenar, en general no se sobreajusta
- **Desventajas:** Supuestos fuertes, características independientes y probabilidades normales
- Podemos entrenar un clasificador para datos discretos si usamos distribuciones multinomiales
- Implementación en [scikit-learn](https://scikit-learn.org/stable/modules/naive_bayes.html)
- [Más sobre clasificador bayesiano](https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html)
